In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')

%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import xgboost as xgb

In [ ]:
# Load Dataset
df = pd.read_csv('df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [ ]:
# without demographics
#df = df.iloc[::,:74]

In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
file_name = 'XGB.60d.1d'
num_companies = 29  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days # 24 time periods per day per stock
total_rows = len(df['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2022-06-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model = XGBRegressor(n_estimators=100, max_depth=100, learning_rate=0.1, objective='reg:squarederror'
                     , alpha=10, tree_method="gpu_hist")

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 16
Count Down: 15
Count Down: 14
Count Down: 13
Count Down: 12
Count Down: 11
Count Down: 10
Count Down: 9
Count Down: 8
Count Down: 7
Count Down: 6
Count Down: 5
Count Down: 4
Count Down: 3
Count Down: 2
Count Down: 1
Count Down: 0


In [ ]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,258.139618,16.520382,4.179022
54,2022-09-01 09:45:00,273.62,257.171936,16.448064,4.179022
83,2022-09-01 10:00:00,271.36,257.569672,13.790328,4.179022
112,2022-09-01 10:15:00,273.14,256.152252,16.987748,4.179022
141,2022-09-01 10:30:00,269.54,255.412140,14.127860,4.179022
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,113.105736,8.354264,4.112352
56285,2022-12-30 14:30:00,121.30,113.138290,8.161710,4.112352
56314,2022-12-30 14:45:00,121.51,114.737747,6.772253,4.112352
56343,2022-12-30 15:00:00,122.42,114.972549,7.447451,4.112352


In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
predicted, actual,duration=result['PREDICTED'],result['ACTUAL'],result['TRAIN_DURATION']

In [ ]:
rmse = np.sqrt(mean_squared_error(result['ACTUAL'], result['PREDICTED']))
# Calculate MAPE %
mape = mean_absolute_percentage_error(result['ACTUAL'], result['PREDICTED']) * 100
# Calculate MPE %
mpe = mean_positive_error(result['ACTUAL'], result['PREDICTED'])
# Calculate MTT in seconds
mtt =duration.sum()

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mtt], index=['	stock_ID','MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
rmse,mape,mpe,metrics_df['MTT'].sum()

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mape, mpe, mtt], index=['stock_ID',	'MAPE',	'MPE',	'MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
#metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,4.021764,2.275390,2.316261,4.098558
1,AMD,3.836275,4.672238,2.237018,4.098558
2,AMZN,6.204470,4.326618,3.104500,4.098558
3,BA,9.378846,4.393012,1.659983,4.098558
4,BABA,6.960532,6.629483,2.416935,4.098558
5,BAC,1.403891,3.182488,0.493789,4.098558
6,C,1.210568,2.165900,0.517129,4.098558
7,CSCO,1.179396,2.009332,0.301837,4.098558
8,CVX,6.297434,3.057929,1.490724,4.098558
9,DIS,4.679561,3.255493,2.203500,4.098558


In [ ]:
avg_values=metrics_df.mean()
avg_df = pd.DataFrame(avg_values).T
avg_df